In [1]:
from interpreter import interpreter
import os

In [4]:
interpreter.llm.model = "azure/gpt-4.1"
interpreter.llm.api_key = os.getenv("AZURE_OPENAI_API_KEY")
interpreter.llm.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
interpreter.llm.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
interpreter.auto_run = True
interpreter.system_message = "You are a helpful assistant, who use maths to answer questions."

output = interpreter.chat("who are you?")

print(output)

[{'role': 'assistant', 'type': 'message', 'content': 'I am a helpful AI assistant designed to answer your questions, solve problems using mathematics, and provide guidance and support. Let me know how I can assist you today!'}]


In [ ]:
message_with_image = [
        {"role": "user", "type": "message", "content": "Describe this image and then, using pyautogui, press ctrl + t to open a new tab in the browser."},
        {"role": "user", "type": "image", "format": "path", "content": image_path}
    ]

In [7]:
import os
import argparse
from datetime import datetime
from typing import Optional, Tuple

# prefer mss (fast) but fallback to Pillow if unavailable
try:
    import mss
    import mss.tools
    _HAS_MSS = True
except Exception:
    _HAS_MSS = False
    try:
        from PIL import ImageGrab  # pillow fallback
    except Exception:
        ImageGrab = None

def capture_screenshot(save_path: Optional[str] = None, region: Optional[Tuple[int, int, int, int]] = None, return_bytes: bool = False):
    """
    Capture a screenshot.
    - save_path: path to save PNG. If None, saves to cwd with timestamp.
    - region: (left, top, width, height) or None for full screen.
    - return_bytes: if True, returns PNG bytes; otherwise returns the save_path string.
    This function avoids using Python 3.10+ union-type syntax so it's
    compatible with older Python versions.
    """
    if save_path is None:
        ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        save_path = os.path.join(os.getcwd(), f"screenshot_{ts}.png")

    if _HAS_MSS:
        with mss.mss() as s:
            monitor = s.monitors[1] if region is None else {
                "left": int(region[0]), "top": int(region[1]),
                "width": int(region[2]), "height": int(region[3])
            }
            s_img = s.grab(monitor)
            mss.tools.to_png(s_img.rgb, s_img.size, output=save_path)
    else:
        # Pillow fallback (Windows/macOS; on some Linux configs may need X server)
        if ImageGrab is None:
            raise RuntimeError("No screenshot backend available: install 'mss' or 'pillow'")
        bbox = None if region is None else (int(region[0]), int(region[1]), int(region[0]) + int(region[2]), int(region[1]) + int(region[3]))
        img = ImageGrab.grab(bbox=bbox)
        img.save(save_path, "PNG")

    if return_bytes:
        with open(save_path, "rb") as f:
            return f.read()
    return save_path

if __name__ == "__main__":
    p = argparse.ArgumentParser(description="Take a screenshot and save as PNG.")
    p.add_argument("--out", "-o", help="Output file path (defaults to cwd with timestamp).")
    p.add_argument("--region", "-r", nargs=4, type=int, metavar=("LEFT", "TOP", "WIDTH", "HEIGHT"),
                   help="Region to capture: left top width height")
    args = p.parse_args()

    path = capture_screenshot(save_path=args.out, region=tuple(args.region) if args.region else None)
    print(path)

usage: ipykernel_launcher.py [-h] [--out OUT] [--region LEFT TOP WIDTH HEIGHT]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\sathish\AppData\Roaming\jupyter\runtime\kernel-v3e054dfdb18a3a7b7d9753810a674c0fd16e96827.json


SystemExit: 2